In [24]:
#import dependencies
import pandas as pd
import numpy as np
from citipy import citipy
import requests
from config import weather_api_key

In [25]:
#Create a set of random latitude and longitude combinations
lats = np.random.uniform(-90,90,size=2000)
lngs = np.random.uniform(-180,180,size=2000)
lat_lngs = zip(lats,lngs)
#Add the latitude and longitude to a list
coordinates = list(lat_lngs)

In [26]:
#Create a list to hold the list of cities
cities =[]
#Identify the nearest city for each lat and long combination
for coordinate in coordinates:
    city= citipy.nearest_city(coordinate[0],coordinate[1]).city_name
    
    #Only append if unique
    if city not in cities:
        cities.append(city)

In [28]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

#declare empty list to store city data
city_data =[]

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_description})
    # If an error is experienced, skip the city.
    except:
        pass

In [29]:
#Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,Hilo,19.7297,-155.0900,75.20,64,75,3.44,US,broken clouds
1,Dikson,73.5069,80.5464,-29.56,86,91,17.40,RU,overcast clouds
2,Hobart,-42.8794,147.3294,66.99,52,75,18.41,AU,broken clouds
3,Ancud,-41.8697,-73.8203,60.01,88,0,5.75,CL,clear sky
4,Ocos,14.5094,-92.1933,71.60,100,20,4.45,GT,few clouds
...,...,...,...,...,...,...,...,...,...
670,Pacific Grove,36.6177,-121.9166,50.00,87,1,6.91,US,clear sky
671,Razole,16.4833,81.8333,80.60,57,56,5.75,IN,haze
672,Ewa Beach,21.3156,-158.0072,75.20,50,1,5.75,US,clear sky
673,Kudahuvadhoo,2.6708,72.8944,81.59,76,0,16.20,MV,clear sky


In [30]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")